## Decision Tree
- usage: classfication and regression
- Can be regarded as determinstic model or probabilistic model
-  A **white box** model: simple to understand and to interpret
- non-parametric model: the number of parameters is not fixed
- effectiveness:
> The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in **an ensemble learner** (random forest), where the features and samples are randomly sampled with replacement. (scikit-learn document)[https://scikit-learn.org/stable/modules/tree.html]
- efficiency:
    * In general, the run time cost to contruct a balanced binary tree is $O(N*n*\log(N))$, where $N$ is the number of samples, and $n$ is the number of features.
    * The cost of using the tree to make prediction $O(\log(N))$
- Three elements in decision tree:
    1. feature selection (ID3, C4.5, CART)
    2. tree training/generation (local optimal)
    3. tree pruning (global optimal)

### ID3 (Iterative Dichotomiser 3)
- [Quinlan 1979](http://hunch.net/~coms-4771/quinlan.pdf)
- key idea: information gain of a feature
- Entropy $H(X)$: $X$ is a discrete random variable, and its prob. distribution is $P(X=x_i)=p_i, \quad i=1,2,\ldots,n$. The entropy of $X$ is: $$H(X) = - \sum_{i=1}^n p_i \log p_i$$
    A larger entropy means a higher degree of uncertainty.
- Conditional Entropy $H(Y|X)$: the expectation of the entropy of conditional prob. $p(Y|X)$ to $X$ ($X$给定条件下$Y$的条件概率分布的熵对$X$的数学期望): $$H(Y|X)=\sum_{i=1}^n p_i H(Y|X=x_i)$$

- **Information gain (IG)**: the IG of a feature $A$ to the training data $D$, denoted as $g(D,A)$, is the difference between $H(D)$ and $H(D|A)$: $$g(D,A) = H(D) - H(D|A) \label{eq:IG} \tag{3}$$
    IG stands for how much the uncertainty of $D$ has been decreased if we know feature $A$.
    
    IG is also the mutual information between the output class (Y) and an input feature.
    
    $D$ is the training data. $|D|$ is $n_{samples}$. $K$ is the number of classes. $|C_k|$ is the number of samples which belong to class $k$. Suppose a discrete feature $A$ with possible values $\{a_1, a_2, \ldots, a_n\}$. Based on the values of $A$, $D$ is partitioned into $n$ subsets $\{D_1, D_2, \ldots, D_n\}$. $|D_{ik}|$ is the number of examples in subset $D_i$ and belonging to class $k$.
    $$H(D) = - \sum_{k=1}^K \frac{|C_k|}{|D|} \log \frac{|C_k|}{|D|} \tag{1}$$
    \begin{align}
    H(D|A) &= \sum_{i=1}^n \frac{|D_i|}{|D|} H(D|A=a_i) \\ &= -\sum_{i=1}^n \frac{|D_i|}{|D|} \sum_{k=1}^K \frac{|D_{ik}|}{|D_i|} \log \frac{|D_{ik}|}{|D_i|} \tag{2}
    \end{align}
- Dealing with continuous input features: use several intervals to partition continuous variable.
    
- **ID3 Algorithm to generate Tree $T$**

    Input: labelled traing data $D$
    
    Output: Tree $T$
    
    1. Create a node $N$
    2. If all the training exmaples belong to a same class, $N$ is the only node in $T$ and label $N$ with the calss. Return $T$ as $N$.
    3. If there are zero input features, $N$ is the only node in $T$ and label $N$ with the majority calss. Return $T$ as $N$.
    4. Otherwise, choose the feature $A$ with the maximum $g(D,A)$ as the partition feature of the current node $N$
    5. For each value $a_i$ in $A$, create a branch from node $N$ with a sub training dataset $D_i$. If $D_i$ is not empty, repeat 4 and 5.

- ID3 limitation: tend to choose features who have a larger number of values

### C4.5
- [Quinlan, 1993](https://en.wikipedia.org/wiki/C4.5_algorithm#cite_note-1)
- improved ID3: 
    1. use IG ratio to choose features
     $$g_R(D,A) = \frac{H(D) - H(D|A)}{H_A(D)} \label{eq:IGR} \tag{4}$$
     where $H_A(D) = -\sum_{i=1}^n \frac{|D_i|}{|D|} \log \frac{|D_i|}{|D|}$, $n$ is the number of different values that feature $A$ has.
    2. handling both continuous and discrete features
    3. training data with missing attribute values
    4. pruning trees after creation
> A landmark decision tree program that is probably the machine learning workhorse most widely used in practice to date. (2011, authors of the Weka machine learning software)

### C5.0
- Quinlan
- both [public source code](https://www.rulequest.com/see5-unix.html) and commercial version are available
- [A number of improvements on C4.5](https://www.rulequest.com/see5-comparison.html)
    1. faster
    2. smaller
    3. boosting (more accurate)
    4. weighting (allows you to weight different cases and misclassification types)
    5. less memory usage
    6. winnowing: winnows the attributes to remove those that may be unhelpful


**ID3 Implementation**

In [1]:
# entropy
# ig

### Pruning Tree
- objective: minimise cost 
    \begin{align}
       C_\alpha(T) &= C(T)+\alpha|T| \\
                   &= \sum_{t=1}^{|T|}|N_t| H_t(T)+\alpha|T| \\
                   &= -\sum_{t=1}^{|T|}|N_t| \sum_{k=1}^{K} \frac{|N_{tk}|}{|N_t|} \log \frac{|N_{tk}|}{|N_t|}+\alpha|T| \\
                   &= -\sum_{t=1}^{|T|}\sum_{k=1}^{K} |N_{tk}|\log \frac{|N_{tk}|}{|N_t|}+\alpha|T|
                   \tag{4}
    \end{align}
    where $|T|$ is the number of leaf nodes in $T$. For a leaf node $t$, the number of samples in it is $N_t$, and $N_{tk}$ is the number of sample in node $t$ and belongs to class $k$.
    
    $C(T)$ is the training error, and $|T|$ stands for the model complexity. $\alpha$ is used to seek a balance between training error and model complexity; a larger $\alpha$ means that we want a more simple model (less leaf nodes). When $\alpha=0$, the optimal tree is the original tree $T$ itself.

- idea: compare the costs between the current tree $T_A$ with the tree $T_B$ which is a part of $T_A$ by cutting some leaf nodes in $T_A$: if $C_\alpha(T_A) \geq C_\alpha(T_B)$, then go with prune.

<img src='images/DT_pruning.png' width='700'>

- **Pruning algorithm for ID3, C4.5**

    Input: Tree $T$ and parameter $\alpha$
    
    Output: Tree after pruning $T_\alpha$ 
    
    1. calculate $H_t(T)$ for every node in the Tree $T$
    2. recursively cut from leaf nodes. Suppose $C_\alpha(T_A)$ and $C_\alpha(T_B)$ are the costs before and after pruning respectively, if $C_\alpha(T_A) \geq C_\alpha(T_B)$, then go with prune.
    3. Repeat 2 until stop, then we can get the subtree $T_\alpha$ whose cost function is the minimum.
    
    limitation: $\alpha$ is decided based on experience.

### CART(Classification and Regression Trees)
- L. Breiman, J. Friedman, R. Olshen, and C. Stone. Classification and Regression Trees. Wadsworth, Belmont, CA, 1984.
